In [1]:
import os
import numpy as np
import pandas as pd
# from sklearn import     # Not sure yet... I don't want a simple multilinear regression, though.


SETUP

In [2]:
#open directory with the spectra, PCA-reduced audio data, and register data in it.
os.chdir('./Training_data/')

SpectrumFeatures = pd.read_csv('000SampleSpectra.csv', index_col = 0) # Should either unstack, or decide which to use.
print(SpectrumFeatures.head())

WavFeatures = pd.read_csv('000PcaReducedWav.csv', index_col = 0)
print(WavFeatures.head())

RegisterTargets = pd.read_csv('000SampleRegisters.csv', index_col = 0)
RegisterTargets = RegisterTargets.drop(['Ratio1', 'Attack1', 'Decay1', 'Sustain1',
                        'Ratio2', 'Attack2', 'Decay2', 'Sustain2',
                        'Ratio3', 'Attack3', 'Decay3', 'Sustain3',
                        'Ratio4', 'Attack4', 'Decay4', 'Sustain4'], axis = 'columns') # Not needed here.
print(RegisterTargets.head())

                                              0            1             2  \
('01 - Circle of Life_0.wav', 'Pwr')  61.014667  1112.303713  1.220439e-02   
('01 - Circle of Life_0.wav', 'dB')   17.854342    30.462234 -1.913484e+01   
('01 - Circle of Life_1.wav', 'Pwr')   0.044072   101.640445  1.006199e+01   
('01 - Circle of Life_1.wav', 'dB')  -13.558345    20.070666  1.002684e+01   
('01 - Circle of Life_2.wav', 'Pwr')   0.000002     0.000109  7.362571e-07   

                                              3          4          5  \
('01 - Circle of Life_0.wav', 'Pwr')   0.003119   0.001861   0.001438   
('01 - Circle of Life_0.wav', 'dB')  -25.059827 -27.301782 -28.422371   
('01 - Circle of Life_1.wav', 'Pwr')  14.333925   3.912804   0.825094   
('01 - Circle of Life_1.wav', 'dB')   11.563651   5.924881  -0.834964   
('01 - Circle of Life_2.wav', 'Pwr')   0.000005   0.000005   0.000005   

                                              6          7          8  \
('01 - Circle of Li

TESTING AN ESTIMATOR MODEL